In [1]:
# netkeiba_race_scrape_and_parse.py
# 目的:
#  1) db.netkeiba.com/race/<race_id>/ を取得してHTML保存（キャッシュ）
#  2) EUC-JP でデコードして文字化け回避
#  3) BeautifulSoupで正規化して pd.read_html でテーブル抽出
#  4) レース結果テーブルを自動特定
#  5) horse_id / jockey_id を href から抽出して DataFrame に付与

import random
import time
import re
from pathlib import Path
from io import StringIO

import requests
import pandas as pd
from bs4 import BeautifulSoup


BASE = "https://db.netkeiba.com"

# 保存先（好きに変えてOK）
DATA_DIR = Path("data_netkeiba")
HTML_DIR = DATA_DIR / "html"
HTML_DIR.mkdir(parents=True, exist_ok=True)

# polite settings（最初は遅く）
SLEEP_MIN = 2.5
SLEEP_MAX = 6.0


def fetch_html(url: str, save_path: Path, session: requests.Session) -> bytes:
    """
    url からHTMLを取得して save_path に保存（存在すれば再取得しない）
    戻り値: HTML bytes
    """
    if save_path.exists():
        return save_path.read_bytes()

    r = session.get(url, timeout=30)
    if r.status_code != 200:
        raise RuntimeError(f"HTTP {r.status_code}: {url}")

    save_path.parent.mkdir(parents=True, exist_ok=True)
    save_path.write_bytes(r.content)

    time.sleep(random.uniform(SLEEP_MIN, SLEEP_MAX))
    return r.content


def decode_netkeiba(html_bytes: bytes) -> str:
    """
    netkeiba系は EUC-JP のことが多いのでまずEUC-JPで読む。
    ダメなら UTF-8 / CP932 も試す。
    """
    for enc in ["euc-jp", "cp932", "utf-8"]:
        try:
            return html_bytes.decode(enc, errors="strict")
        except UnicodeDecodeError:
            continue
    # 最後の保険（多少欠けてもいいから進める）
    return html_bytes.decode("euc-jp", errors="ignore")


def make_soup(html: str) -> BeautifulSoup:
    """
    BeautifulSoup を作る。lxml が使えなければ html.parser にフォールバック。
    """
    try:
        return BeautifulSoup(html, "lxml")  # ← ここは "lxml"（lmxlじゃない）
    except Exception:
        return BeautifulSoup(html, "html.parser")


def read_all_tables(soup: BeautifulSoup) -> list[pd.DataFrame]:
    """
    soup から pandas の read_html で全テーブルを抜く。
    FutureWarning回避のため StringIO を使う。
    """
    return pd.read_html(StringIO(str(soup)))


def is_race_result_table(df: pd.DataFrame) -> bool:
    """
    レース結果っぽいテーブルかどうか判定。
    netkeibaは列名が崩れることもあるので、複数条件でゆるく判定する。
    """
    cols = [str(c).strip() for c in df.columns]
    colset = set(cols)

    # よくある列名
    must_have_any = [{"着順", "順位"}, {"馬名"}, {"騎手"}]
    hits = 0
    for s in must_have_any:
        if len(colset.intersection(s)) > 0:
            hits += 1

    # 行数がそこそこある & 主要列がそこそこ揃ってる
    return (df.shape[0] >= 5) and (hits >= 2)


def normalize_header(df: pd.DataFrame) -> pd.DataFrame:
    """
    Unnamed地獄対策：
    もし列名に「着順」「馬名」などが入っていなければ
    先頭行がヘッダの可能性があるので繰り上げる。
    """
    cols = set(map(str, df.columns))
    if ("馬名" in cols) or ("着順" in cols) or ("順位" in cols):
        return df.reset_index(drop=True)

    # 先頭行をヘッダにしてみる
    df2 = df.copy()
    df2.columns = df2.iloc[0]
    df2 = df2.iloc[1:].reset_index(drop=True)
    return df2


def extract_ids(soup: BeautifulSoup) -> tuple[dict[str, str], dict[str, str]]:
    """
    aタグの href から horse_id / jockey_id を抽出して
    表示テキスト（馬名/騎手名）→ID の辞書を返す。
    """
    horse_map: dict[str, str] = {}
    jockey_map: dict[str, str] = {}

    for a in soup.select("a[href]"):
        text = a.get_text(strip=True)
        href = a.get("href", "")

        # horse
        if "/horse/" in href:
            m = re.search(r"/horse/(\d+)/", href)
            if m and text:
                # 同名が出る場合があるので「最初に見つかったもの」を優先
                horse_map.setdefault(text, m.group(1))

        # jockey
        if "/jockey/" in href:
            m = re.search(r"/jockey/(\d+)/", href)
            if m and text:
                jockey_map.setdefault(text, m.group(1))

    return horse_map, jockey_map


def parse_race_page(html_bytes: bytes) -> pd.DataFrame:
    """
    レースページHTML(bytes) → 結果テーブル(DataFrame) を返す
    horse_id/jockey_id も付与。
    """
    html = decode_netkeiba(html_bytes)
    soup = make_soup(html)

    tables = read_all_tables(soup)
    if not tables:
        raise RuntimeError("No tables found. The page structure might have changed.")

    # 結果っぽいテーブルを探す
    cand = [t for t in tables if is_race_result_table(t)]
    if not cand:
        # だめなら「一番行数が多いテーブル」を候補にする
        df = max(tables, key=lambda x: x.shape[0])
    else:
        # 候補の中で最も行数が多いものを採用
        df = max(cand, key=lambda x: x.shape[0])

    df = normalize_header(df)

    # ID抽出
    horse_map, jockey_map = extract_ids(soup)

    # 列名ゆらぎ対策
    if "馬名" in df.columns:
        df["horse_id"] = df["馬名"].astype(str).map(horse_map)
    else:
        df["horse_id"] = None

    if "騎手" in df.columns:
        df["jockey_id"] = df["騎手"].astype(str).map(jockey_map)
    else:
        df["jockey_id"] = None

    return df


def get_race_df(race_id: str) -> pd.DataFrame:
    """
    race_id を指定して、HTML取得→パース→DataFrame を返す。
    """
    url = f"{BASE}/race/{race_id}/"
    save_path = HTML_DIR / "race" / f"{race_id}.html"

    session = requests.Session()
    session.headers.update({
        "User-Agent": "Mozilla/5.0 (compatible; research-bot/0.1; +local)",
        "Accept-Language": "ja,en;q=0.8",
    })

    html_bytes = fetch_html(url, save_path, session)
    df = parse_race_page(html_bytes)
    df.insert(0, "race_id", race_id)
    return df


if __name__ == "__main__":
    # 例：任意の race_id を入れて動作確認
    # race_id はあなたが取得したいレースのIDに置き換えてください
    race_id = "202401010101"  # ダミー（ここを実在IDに）
    df = get_race_df(race_id)

    # 表示
    print(df.head())



        race_id  着 順  枠 番  馬 番         馬名  性齢  斤量    騎手     タイム     着差    単勝  \
0  202401010101    1    5    5    ポッドベイダー  牡2  55  佐々木大  1:08.8    NaN   1.2   
1  202401010101    2    2    2  ニシノクードクール  牝2  55   武藤雅  1:09.1  1.3/4  10.2   
2  202401010101    3    3    3    ロードヴェルト  牡2  55  横山武史  1:09.4  1.3/4   7.9   
3  202401010101    4    1    1   ルージュアマリア  牝2  55  永野猛蔵  1:10.0  3.1/2   5.9   
4  202401010101    5    4    4   ロードヴァルカン  牡2  54  角田大河  1:10.1     クビ  21.3   

   人 気      馬体重       調教師    horse_id  jockey_id  
0    1  462(-2)  [東] 上原佑紀  2022105244        NaN  
1    4  452(-2)  [東] 武藤善則  2022106999        NaN  
2    3  416(+6)  [西] 牧浦充徳  2022100639        NaN  
3    2  410(+6)  [東] 黒岩陽一  2022105762        NaN  
4    5  438(-2)  [西] 中村直也  2022100660        NaN  


# セル1：開催日（YYYYMMDD）→その日の race_id を全部集める

In [2]:
# --- セル1（最終版）: スマホ版レース一覧から race_id を確実に抜く ---

import re
import time
import random
from datetime import date, timedelta
from typing import List, Set

import requests

# スマホ版（HTMLに情報が残りやすい）
RACE_LIST_URL = "https://race.sp.netkeiba.com/?pid=race_list&kaisai_date={yyyymmdd}"

SLEEP_MIN = 0.8
SLEEP_MAX = 2.0

sess = requests.Session()
sess.headers.update({
    # スマホっぽいUAにして「ブラウザで見えてるHTML」に寄せる
    "User-Agent": (
        "Mozilla/5.0 (iPhone; CPU iPhone OS 17_0 like Mac OS X) "
        "AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.0 Mobile/15E148 Safari/604.1"
    ),
    "Accept-Language": "ja,en;q=0.8",
})

def yyyymmdd(d: date) -> str:
    return d.strftime("%Y%m%d")

def get_race_ids_for_date(d: date, debug: bool = False) -> List[str]:
    """
    【何をしている？】
    - スマホ版のレース一覧ページを取得
    - HTML “全文” から race_id=12桁 を全部抜く（hrefに依存しない）
    - 重複を除いて race_id リストを返す
    """
    url = RACE_LIST_URL.format(yyyymmdd=yyyymmdd(d))
    r = sess.get(url, timeout=30)

    if r.status_code != 200:
        if debug:
            print("[skip]", d, "status", r.status_code, "url", url)
        return []

    # 文字化け回避（netkeibaはEUC-JP系が混ざることがある）
    r.encoding = r.apparent_encoding
    html = r.text

    # hrefだけを見ると取り逃がすことがあるので、HTML全体から抜く
    ids = sorted(set(re.findall(r"race_id=(\d{12})", html)))

    if debug:
        print("[date]", d, "url", url, "status", r.status_code, "ids", len(ids))
        print("  sample ids:", ids[:5])
        # 0件の時は「本当にrace_idが無いHTML」か確認
        if len(ids) == 0:
            print("  debug: 'race_id=' count =", html.count("race_id="))
            print("  head:", html[:500])

    time.sleep(random.uniform(SLEEP_MIN, SLEEP_MAX))
    return ids

def get_race_ids_in_range(start: date, end: date, debug: bool = False) -> List[str]:
    """
    【何をしている？】
    - start〜end（両端含む）を日ごとに回して race_id を集約
    """
    all_ids: Set[str] = set()
    d = start
    while d <= end:
        ids = get_race_ids_for_date(d, debug=debug)
        all_ids.update(ids)
        d += timedelta(days=1)
    return sorted(all_ids)


In [12]:
# デバッグ用：1日だけHTMLを確認
d = end - timedelta(days=7)
url = RACE_LIST_URL.format(yyyymmdd=d.strftime("%Y%m%d"))
r = sess.get(url, timeout=30)
r.encoding = r.apparent_encoding

print("status:", r.status_code)
print(r.text[:2000])  # 先頭だけ表示


status: 200


<!DOCTYPE html>
<html>
<head>
<meta charset="EUC-JP">

<!-- block=meta_tag_common_race (d) -->
<meta http-equiv="content-language" content="ja">
<meta name="viewport" content="width=device-width, initial-scale=1.0, maximum-scale=1.0, minimum-scale=0.5, user-scalable=yes">

<meta name="format-detection" content="telephone=no" />
<meta name="description" content="JRA開催レースのレース一覧です。JRA開催レースの出馬表や最新オッズ、レース結果速報、払戻情報をはじめ、競馬予想やデータ分析など予想に役立つ情報も満載です。" />
<meta name="keywords" content="競馬,keiba,出馬表,オッズ,予想,レース結果,払戻し,結果速報,競馬予想,ネット競馬,netkeiba" />
<meta name="thumbnail" content="https://www.netkeiba.com/style/netkeiba.ja/image/netkeiba.png" />
<!-- ogp用 -->
<meta property="og:site_name" content="netkeiba" />
<meta property="og:type" content="article" />
<meta property="og:title" content="レース一覧 | レース情報(JRA) - netkeiba" />
<meta property="og:url" content="https://race.netkeiba.com/top/race_list.html" />
<meta property="og:description" content="JRA開催レースのレース一覧です。JRA開催レースの出馬表や最新オッズ、レース結果速報、払戻

In [13]:
from datetime import date
test_day = date(2025, 12, 28)
ids = get_race_ids_for_date(test_day)
print(len(ids), ids[:5])


48 ['202506050701', '202506050702', '202506050703', '202506050704', '202506050705']


# セル2：過去N年分の race_id を集める（まずは1〜2年で試す）

In [14]:
from datetime import date, timedelta

# 例：まずは過去365日（1年）で試す → 動いたら 2年(730日)、3年…と増やす
end = date.today()
start = end - timedelta(days=365)

race_ids = get_race_ids_in_range(start, end)
print("num race_ids:", len(race_ids))
print("sample:", race_ids[:10])


num race_ids: 3455
sample: ['202501010101', '202501010102', '202501010103', '202501010104', '202501010105', '202501010106', '202501010107', '202501010108', '202501010109', '202501010110']


In [3]:
from datetime import date, timedelta
from pathlib import Path
import pandas as pd

# =========================
# 過去N日分の race_id を取得
# =========================
end = date.today()
start = end - timedelta(days=365)  # ← ここを 730, 1095 にすれば年数拡張

race_ids = get_race_ids_in_range(start, end)

print("num race_ids:", len(race_ids))
print("sample:", race_ids[:10])

# =========================
# ここから「保存」処理（追加）
# =========================
DATA_DIR = Path("data")
DATA_DIR.mkdir(exist_ok=True)

race_ids = (
    pd.Series(race_ids, name="race_id")
      .astype(str)
      .str.zfill(12)
      .drop_duplicates()
      .sort_values()
      .reset_index(drop=True)
)

race_ids.to_csv(DATA_DIR / "race_ids.csv", index=False, encoding="utf-8-sig")
print("[saved] data/race_ids.csv")


num race_ids: 3455
sample: ['202501010101', '202501010102', '202501010103', '202501010104', '202501010105', '202501010106', '202501010107', '202501010108', '202501010109', '202501010110']
[saved] data/race_ids.csv


# セル3：race_id を回して “学習用の生データ(train_raw.csv)” を作る

In [4]:
import pandas as pd

def build_train_raw_from_ids(race_ids, out_csv="train_raw.csv"):
    """
    【何をしている？】
    - 入力: race_idリスト
    - 1つずつ get_race_df(race_id) を実行して結果表を取得
    - 全部結合して out_csv に保存
    """
    dfs = []
    for i, rid in enumerate(race_ids, 1):
        try:
            d = get_race_df(rid)  # ← ここはあなたの既存関数
            dfs.append(d)
            if i % 50 == 0:
                print(f"[progress] {i}/{len(race_ids)} races")
        except Exception as e:
            print("[skip]", rid, e)

    if not dfs:
        raise RuntimeError("No races collected. race_ids might be empty or blocked.")

    df_all = pd.concat(dfs, ignore_index=True)
    df_all.to_csv(out_csv, index=False, encoding="utf-8-sig")
    print("[saved]", out_csv, "shape:", df_all.shape)
    return df_all

df_all = build_train_raw_from_ids(race_ids, out_csv="train_raw.csv")
df_all.head()


[progress] 50/3455 races
[progress] 100/3455 races
[progress] 150/3455 races
[progress] 200/3455 races
[progress] 250/3455 races
[progress] 300/3455 races
[progress] 350/3455 races
[progress] 400/3455 races
[progress] 450/3455 races
[progress] 500/3455 races
[progress] 550/3455 races
[progress] 600/3455 races
[progress] 650/3455 races
[progress] 700/3455 races
[progress] 750/3455 races
[progress] 800/3455 races
[progress] 850/3455 races
[progress] 900/3455 races
[progress] 950/3455 races
[progress] 1000/3455 races
[progress] 1050/3455 races
[progress] 1100/3455 races
[progress] 1150/3455 races
[progress] 1200/3455 races
[progress] 1250/3455 races
[progress] 1300/3455 races
[progress] 1350/3455 races
[progress] 1400/3455 races
[skip] 202506010604 list index out of range
[progress] 1450/3455 races
[skip] 202506010704 list index out of range
[progress] 1500/3455 races
[progress] 1550/3455 races
[progress] 1600/3455 races
[progress] 1650/3455 races
[progress] 1700/3455 races
[progress] 175

,race_id,着 順,枠 番,馬 番,馬名,性齢,斤量,騎手,タイム,着差,単勝,人 気,馬体重,調教師,horse_id,jockey_id
0,202501010101,1,3,3,ショウナンカリス,牝2,55.0,北村友一,1:08.9,NaN,4.8,3.0,408(+4),[東] 加藤士津,2023105347,NaN
1,202501010101,2,4,4,ファニーバニー,牝2,55.0,鮫島克駿,1:09.1,1.1/4,2.4,1.0,428(+4),[西] 杉山佳明,2023105277,NaN
2,202501010101,3,2,2,ハルヒメ,牝2,52.0,小林美駒,1:09.1,ハナ,4.2,2.0,388(-14),[東] 矢嶋大樹,2023103400,NaN
3,202501010101,4,6,7,カンレイスター,牝2,55.0,荻野琢真,1:09.9,5,46.9,8.0,434(-8),[東] 矢野英一,2023105558,NaN
4,202501010101,5,8,12,ココソラーレ,牝2,52.0,鷲頭虎太,1:10.1,1.1/4,77.1,10.0,424(-4),[東] 天間昭一,2023109031,NaN


In [3]:

import pandas as pd

race_ids = pd.read_csv("train_raw.csv")["race_id"].astype(str).str.zfill(12)

done = set(df["race_id"].astype(str).str.zfill(12).unique())
todo = [rid for rid in race_ids if rid not in done]

print("planned races:", len(race_ids))
print("done races:", len(done))
print("remaining races:", len(todo))
print("remaining sample:", todo[:10])


NameError: name 'df' is not defined